### 1. Importing Library

In [58]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.rc("font", size=14)
plt.rcParams['axes.grid'] = True
plt.figure(figsize=(6,3))
plt.gray()

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.preprocessing import  PolynomialFeatures, KBinsDiscretizer, FunctionTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer, OrdinalEncoder
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
#from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz, export
from sklearn.ensemble import BaggingClassifier, BaggingRegressor,RandomForestClassifier,RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor, AdaBoostClassifier, AdaBoostRegressor 
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

<Figure size 432x216 with 0 Axes>

### 2. Loading the dataset

In [3]:
df=pd.read_csv('creditcard.csv')
df.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
df.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0
284806,172792.0,-0.533413,-0.189733,0.703337,-0.506271,-0.012546,-0.649617,1.577006,-0.414650,0.486180,...,0.261057,0.643078,0.376777,0.008797,-0.473649,-0.818267,-0.002415,0.013649,217.00,0


1. Lets understand the data now so the data has features such as time (secs),amount (USD), class and V1-V28 are transactions.
2. The time columns give us the elapsed time of every transaction from the first transaction.
3. V1-V28 features of every transaction as the data is of credit card the feature has been converted to numerical values through principal component analysis.
4. Class column gives us the details about the transaction is fraud or legit as '0' represent the transaction is legit and '1' represent the transaction is fraud.

### 3. Understand the Data

In [5]:
df.info()  # we have no missing value in our dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [14]:
df.isnull().sum()

V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [6]:
# We have one feature 'time' which has no effect in our prediction so we can drop that column

df.drop(['Time'],axis=1, inplace= True)

In [7]:
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [9]:
# We have to check for any duplicate data and drop it.
df.shape

(284807, 30)

In [10]:
df.drop_duplicates(inplace=True)

In [11]:
df.shape 

(275663, 30)

In [12]:
# so the duplicate value which was 9144 values was droped. Now we have cleaned data.

In [15]:
df.describe()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,275663.000000,275663.000000,275663.000000,275663.000000,275663.000000,275663.000000,275663.000000,275663.000000,275663.000000,275663.000000,...,275663.000000,275663.000000,275663.000000,275663.000000,275663.000000,275663.000000,275663.000000,275663.000000,275663.000000,275663.000000
mean,-0.037460,-0.002430,0.025520,-0.004359,-0.010660,-0.014206,0.008586,-0.005698,-0.012363,0.003114,...,0.002579,0.005827,-0.001941,-0.006868,-0.004812,-0.000240,0.001921,0.000904,90.578380,0.001716
std,1.952522,1.667260,1.507538,1.424323,1.378117,1.313213,1.240348,1.191596,1.100108,1.087025,...,0.733089,0.726378,0.631451,0.605550,0.524175,0.484139,0.401271,0.332649,253.213954,0.041388
min,-56.407510,-72.715728,-48.325589,-5.683171,-113.743307,-26.160506,-43.557242,-73.216718,-13.434066,-24.588262,...,-34.830382,-10.933144,-44.807735,-2.836627,-10.295397,-2.604551,-22.565679,-15.430084,0.000000,0.000000
25%,-0.941105,-0.614040,-0.843168,-0.862847,-0.700192,-0.765861,-0.552047,-0.209618,-0.659904,-0.538968,...,-0.225021,-0.532173,-0.165440,-0.361062,-0.323597,-0.328290,-0.071729,-0.052654,6.300000,0.000000
50%,-0.059659,0.070249,0.200736,-0.035098,-0.060556,-0.270931,0.044848,0.022980,-0.064724,-0.091752,...,-0.025637,0.013397,-0.013655,0.037569,0.009909,-0.056667,0.002615,0.011788,23.740000,0.000000
75%,1.294471,0.819067,1.048461,0.753943,0.604521,0.387704,0.583885,0.322319,0.593098,0.470702,...,0.189118,0.534272,0.145482,0.432931,0.347151,0.244196,0.094730,0.081355,79.910000,0.000000
max,2.454930,22.057729,9.382558,16.875344,34.801666,73.301626,120.589494,20.007208,15.594995,23.745136,...,27.202839,10.503090,22.528412,4.584549,7.519589,3.517346,31.612198,33.847808,25691.160000,1.000000


In [16]:
round(100* (df.isnull().sum()/len(df)),2).sort_values(ascending=False)

V1        0.0
V2        0.0
Amount    0.0
V28       0.0
V27       0.0
V26       0.0
V25       0.0
V24       0.0
V23       0.0
V22       0.0
V21       0.0
V20       0.0
V19       0.0
V18       0.0
V17       0.0
V16       0.0
V15       0.0
V14       0.0
V13       0.0
V12       0.0
V11       0.0
V10       0.0
V9        0.0
V8        0.0
V7        0.0
V6        0.0
V5        0.0
V4        0.0
V3        0.0
Class     0.0
dtype: float64

In [17]:
#let check the class column 

df['Class'].value_counts()

0    275190
1       473
Name: Class, dtype: int64

0 - Legit transaction
1 - Fraud transaction

We can see the data is highly unbalanced.
As most of the data is in legit transaction and if we feed this data to machine learning model it will always predict normal transaction only.

In [18]:
# seprating the class data 

legit=df[df.Class == 0]
fraud=df[df.Class == 1]

In [19]:
print(legit.shape)
print(fraud.shape)

(275190, 30)
(473, 30)


In [22]:
sc = StandardScaler()
amount = df['Amount'].values
df['Amount'] = sc.fit_transform(amount.reshape(-1, 1))

In [23]:
legit.Amount.describe()

count    275190.000000
mean         90.521154
std         253.198478
min           0.000000
25%           6.390000
50%          23.750000
75%          79.900000
max       25691.160000
Name: Amount, dtype: float64

In [24]:
fraud.Amount.describe()

count     473.000000
mean      123.871860
std       260.211041
min         0.000000
25%         1.000000
50%         9.820000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [26]:
#compare both the transaction

df.groupby('Class').mean()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,-0.029792,-0.008288,0.037131,-0.012054,-0.005596,-0.011768,0.017497,-0.007346,-0.008050,0.012492,...,0.002717,0.001781,0.005689,-0.001779,-0.006696,-0.004890,-0.000327,0.001557,0.000771,-0.000226
1,-4.498280,3.405965,-6.729599,4.472591,-2.957197,-1.432518,-5.175912,0.953255,-2.522124,-5.453274,...,0.405043,0.466550,0.086639,-0.096464,-0.106643,0.040615,0.050456,0.213774,0.078270,0.131484


### 4. Handling Unbalanced Data

In [31]:
# We use under-sampling technique to handle the unbalanced data. This will give us even distribution of the data

legit_sample=legit.sample(n=473) #473 because the number of fraud transaction is 473

In [32]:
df1=pd.concat([legit_sample,fraud],axis=0)

In [33]:
df1.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
234742,1.942385,-0.138190,-1.824516,0.882982,0.785795,0.069265,0.246293,-0.114157,0.082046,0.469155,...,0.265361,0.718535,-0.243167,-1.436725,0.428976,-0.362596,-0.013621,-0.064349,73.75,0
121924,-1.664083,-2.075529,-0.019806,-0.774787,-2.768038,0.808134,3.614374,-0.539539,-2.069651,-0.634151,...,0.047756,-0.976536,2.152828,-0.193979,-0.187158,0.727950,-0.244753,0.198466,924.27,0
4665,-2.648948,0.907329,1.576313,0.451743,-0.802008,-1.099054,-0.562804,0.755144,1.094435,-0.696071,...,-0.189692,-0.351651,0.287304,0.646668,0.236966,0.200817,-0.178660,0.211711,9.99,0
176838,2.045910,0.183333,-2.433293,0.528798,0.837151,-1.695295,1.212184,-0.779194,-0.385296,0.203293,...,0.308867,1.038821,-0.268046,0.037290,0.733449,0.718944,-0.124407,-0.081422,59.00,0
265340,2.007679,-0.170746,-1.288866,0.282107,-0.120423,-1.041167,0.051572,-0.159438,0.544599,0.215592,...,0.294228,0.908976,0.018215,0.016559,0.162281,-0.123856,-0.030160,-0.070893,5.49,0


In [34]:
df1['Class'].value_counts() #here we have uniformly distributed data 

0    473
1    473
Name: Class, dtype: int64

In [35]:
df1.groupby('Class').mean()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,-0.103452,-0.069147,-0.013513,-0.016560,0.056824,-0.147620,-0.045283,0.003445,-0.012050,-0.007427,...,0.071385,0.011066,0.011965,-0.028267,0.016290,-0.014977,-0.013757,-0.008606,-0.020824,92.08277
1,-4.498280,3.405965,-6.729599,4.472591,-2.957197,-1.432518,-5.175912,0.953255,-2.522124,-5.453274,...,0.405043,0.466550,0.086639,-0.096464,-0.106643,0.040615,0.050456,0.213774,0.078270,123.87186


### 5. Splitting the data into features and target

In [37]:
X=df1.drop(columns='Class',axis=1)
Y=df1['Class']

In [38]:
print(X)

              V1        V2        V3        V4        V5        V6        V7  \
234742  1.942385 -0.138190 -1.824516  0.882982  0.785795  0.069265  0.246293   
121924 -1.664083 -2.075529 -0.019806 -0.774787 -2.768038  0.808134  3.614374   
4665   -2.648948  0.907329  1.576313  0.451743 -0.802008 -1.099054 -0.562804   
176838  2.045910  0.183333 -2.433293  0.528798  0.837151 -1.695295  1.212184   
265340  2.007679 -0.170746 -1.288866  0.282107 -0.120423 -1.041167  0.051572   
...          ...       ...       ...       ...       ...       ...       ...   
279863 -1.927883  1.125653 -4.518331  1.749293 -1.566487 -2.010494 -0.882850   
280143  1.378559  1.289381 -5.004247  1.411850  0.442581 -1.326536 -1.413170   
280149 -0.676143  1.126366 -2.213700  0.468308 -1.120541 -0.003346 -2.234739   
281144 -3.113832  0.585864 -5.399730  1.817092 -0.840618 -2.943548 -2.208002   
281674  1.991976  0.158476 -2.583441  0.408670  1.151147 -0.096695  0.223050   

              V8        V9       V10  .

In [39]:
print(Y)

234742    0
121924    0
4665      0
176838    0
265340    0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 946, dtype: int64


### 6. Train/Test Split

In [40]:
X_train,X_test,Y_train,Y_test= train_test_split(X,Y,test_size=0.25, stratify=Y, random_state=1)

In [41]:
print(X.shape,X_train.shape,X_test.shape)

(946, 29) (709, 29) (237, 29)


### 7. Model Development

#### 1. Logistic Regression

In [42]:
# training the model

model= LogisticRegression()
model.fit(X_train,Y_train)

C:\Users\Asad Khan\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

### 8. Model Evaluation

In [43]:
X_train_prediction=model.predict(X_train)
train_accuracy=accuracy_score(X_train_prediction, Y_train)

In [44]:
print('Accuracy on training data :', train_accuracy)

Accuracy on training data : 0.9576868829337094


In [45]:
# 94.1 prediction score means our model can predict 94 correct prediction out of 100 sample

In [46]:
X_test_prediction=model.predict(X_test)
test_accuracy=accuracy_score(X_test_prediction,Y_test)

In [47]:
print('Accuracy on training data :', test_accuracy)

Accuracy on training data : 0.9493670886075949


In [55]:
print('F1 score of the Decision Tree model is {}'.format(f1_score(Y_test, X_test_prediction)))

F1 score of the Decision Tree model is 0.9482758620689655


In [54]:
confusion_matrix(Y_test, X_test_prediction, labels = [0, 1])

array([[115,   3],
       [  9, 110]], dtype=int64)

#### 2. Decision Tree

In [48]:
dt_train_prediction=DecisionTreeClassifier(max_depth=4, criterion = 'entropy')
dt_train_prediction.fit(X_train, Y_train)
dt_yhat=dt_train_prediction.predict(X_test)

In [50]:
print('Accuracy score of the Decision Tree model is {}'.format(accuracy_score(Y_test, dt_yhat)))

Accuracy score of the Decision Tree model is 0.9240506329113924


In [51]:
print('F1 score of the Decision Tree model is {}'.format(f1_score(Y_test, dt_yhat)))

F1 score of the Decision Tree model is 0.9203539823008848


In [52]:
confusion_matrix(Y_test, dt_yhat, labels = [0, 1])

array([[115,   3],
       [ 15, 104]], dtype=int64)

In [53]:
# So the first row represent positive value where as the second row represents negative. 
# So in total we have 115 positive value and 3 negative value so only out of 118 total transaction only 3 where classified 
# as false prediction.


#### 3. K- Nearest Neighbors

In [60]:
n=7
Knn=KNeighborsClassifier(n_neighbors=n)
Knn.fit(X_train, Y_train)
knn_yhat=Knn.predict(X_test)


In [61]:
print('Accuracy score of the K-Nearest Neighbors model is {}'.format(accuracy_score(Y_test, knn_yhat)))


Accuracy score of the K-Nearest Neighbors model is 0.8523206751054853


In [62]:
print('F1 score of the K-Nearest Neighbors model is {}'.format(f1_score(Y_test, knn_yhat)))

F1 score of the K-Nearest Neighbors model is 0.8471615720524018


In [63]:
confusion_matrix(Y_test, knn_yhat, labels = [0, 1])

array([[105,  13],
       [ 22,  97]], dtype=int64)

#### 4. XGBoost

In [64]:
xgb=XGBClassifier(max_depth=4)
xgb.fit(X_train, Y_train)
xgb_yhat=xgb.predict(X_test)

C:\Users\Asad Khan\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:38:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [65]:
print('Accuracy score of the XGBoost model is {}'.format(accuracy_score(Y_test, xgb_yhat)))

Accuracy score of the XGBoost model is 0.9409282700421941


In [66]:
print('F1 score of the XGBoost model is {}'.format(f1_score(Y_test, xgb_yhat)))

F1 score of the XGBoost model is 0.9401709401709402


In [67]:
confusion_matrix(Y_test, xgb_yhat, labels = [0, 1])

array([[113,   5],
       [  9, 110]], dtype=int64)

#### 5. Random Forest

In [68]:
rf=RandomForestClassifier(max_depth=4)
rf.fit(X_train, Y_train)
rf_yhat=rf.predict(X_test)

In [69]:
print('Accuracy score of the Random Forest model is {}'.format(accuracy_score(Y_test, rf_yhat)))

Accuracy score of the Random Forest model is 0.9367088607594937


In [70]:
print('F1 score of the Random Forest model is {}'.format(f1_score(Y_test, rf_yhat)))

F1 score of the Random Forest model is 0.9333333333333333


In [71]:
confusion_matrix(Y_test, rf_yhat, labels = [0, 1])

array([[117,   1],
       [ 14, 105]], dtype=int64)